In [1]:
# remove this
import sys
#sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

In [2]:
# cleanup any old results
!./cleanup.sh

In [3]:
# Ascent imports
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helper we use when displaying results in the notebook
img_display_width = 500

In [4]:
# create example mesh using conduit blueprint
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      25,
                                      25,
                                      25,
                                      mesh)

In [5]:
# make sure the mesh we created conforms to the blueprint
verify_info = conduit.Node()
if not conduit.blueprint.mesh.verify(mesh,verify_info):
    print("Mesh Verify failed!")
    print(verify_info)
else:
    print("Mesh verify success!")

Mesh verify success!


## Relay Extract Example: Save input mesh and all fields to Blueprint HDF5 Files

In [6]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_all_fields"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"


# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()


- 
  action: "add_extracts"
  extracts: 
    e1: 
      type: "relay"
      params: 
        path: "out_export_braid_all_fields"
        protocol: "blueprint/mesh/hdf5"



## Look at the Blueprint HDF5 extract with VisIt

In [7]:
!visit -o out_export_braid_all_fields.cycle_000100.root

/bin/sh: visit: command not found


## Relay Extract Example: Save input mesh and selected field to Blueprint HDF5 Files

In [8]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_one_field"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

#
# you can also limit the export to only passed field names
#
extracts["e1/params/fields"].append().set("braid")

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()


- 
  action: "add_extracts"
  extracts: 
    e1: 
      type: "relay"
      params: 
        path: "out_export_braid_one_field"
        protocol: "blueprint/mesh/hdf5"
        fields: 
          - "braid"



## Look at the Blueprint HDF5 extract with VisIt

In [9]:
!visit -o out_export_braid_one_field.cycle_000100.root

/bin/sh: visit: command not found


## Relay Extract Example: Save contour result to Blueprint HDF5 Files

In [10]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

pipelines = conduit.Node()

pipelines["pl1/f1/type"] = "contour"
contour_params = conduit.Node()
contour_params["field"] = "braid"

iso_vals = np.array([0.2, 0.4], dtype=np.float64)
contour_params["iso_values"].set_external(iso_vals)
pipelines["pl1/f1/params"] = contour_params

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_pipelines";
add_act["pipelines"] = pipelines;

extracts = conduit.Node()
extracts["e1/type"] = "relay"
extracts["e1/pipeline"]  = "pl1"
extracts["e1/params/path"] = "out_extract_braid_contour"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

# setup actions
add_act2 = actions.append()
add_act2["action"] = "add_extracts"
add_act2["extracts"] = extracts

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()


- 
  action: "add_pipelines"
  pipelines: 
    pl1: 
      f1: 
        type: "contour"
        params: 
          field: "braid"
          iso_values: [0.2, 0.4]
- 
  action: "add_extracts"
  extracts: 
    e1: 
      type: "relay"
      pipeline: "pl1"
      params: 
        path: "out_extract_braid_contour"
        protocol: "blueprint/mesh/hdf5"



## Look at the HDF5 extract with VisIt

In [11]:
!visit -o out_extract_braid_contour.cycle_000100.root

/bin/sh: visit: command not found


## Cinema Extract Example: Create a Cinema Image Database of a contour

In [12]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

pipelines = conduit.Node()
pipelines["pl1/f1/type"] = "contour"

contour_params = conduit.Node()
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set_external(iso_vals)
pipelines["pl1/f1/params"] = contour_params

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
add_act["pipelines"] = pipelines

# declare a scene to render the dataset
scenes = conduit.Node()
scenes["s1/plots/p1/type"] = "pseudocolor";
scenes["s1/plots/p1/pipeline"] = "pl1";
scenes["s1/plots/p1/field"] = "braid";
# cinema params
scenes["s1/renders/r1/type"] = "cinema"
scenes["s1/renders/r1/phi"] = 5
scenes["s1/renders/r1/theta"] = 5
scenes["s1/renders/r1/db_name"] = "out_extract_cinema"

add_act2 = actions.append()
add_act2["action"] = "add_scenes"
add_act2["scenes"] = scenes

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()


- 
  action: "add_pipelines"
  pipelines: 
    pl1: 
      f1: 
        type: "contour"
        params: 
          field: "braid"
          iso_values: [0.200000002980232, 0.400000005960464]
- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "pseudocolor"
          pipeline: "pl1"
          field: "braid"
      renders: 
        r1: 
          type: "cinema"
          phi: 5
          theta: 5
          db_name: "out_extract_cinema"



In [13]:
from IPython.display import HTML
HTML(filename="cinema_databases/out_extract_cinema/index.html")